In [5]:
%%capture

!pip install cudaq

In [6]:
import cudaq

import os
from getpass import getpass

api_key = os.getenv('IONQ_API_KEY') or getpass('Enter your IonQ API key: ')
os.environ['IONQ_API_KEY'] = api_key  # Remove when set_target takes api_key
cudaq.set_target('ionq', qpu='simulator')

Enter your IonQ API key: ··········


In [ ]:
# Define the kernel and allocate two qubits
kernel = cudaq.make_kernel()
qubits = kernel.qalloc(2)

# Apply a Hadamard gate to the first qubit
kernel.h(qubits[0])

# Apply a CX gate from the first to second qubit
kernel.cx(qubits[0], qubits[1])

# Execute the kernel on the computer and print the result
print(cudaq.sample(kernel))

{ 3:50 0:50 }



In [7]:
import cudaq

# Define a circuit (CUDA-Q kernel)
@cudaq.kernel
def hello_world():
    qubits = cudaq.qvector(2)
    h(qubits[0])
    x.ctrl(qubits[0], qubits[1])

# Set the target to IonQ's simulator
cudaq.set_target('ionq')

# Run the circuit and print results
result = cudaq.sample(hello_world)
print(result)

{ 00:500 11:500 }



In [ ]:
import cudaq

# Define a circuit (CUDA-Q kernel)
@cudaq.kernel
def hello_qpu():
    qubits = cudaq.qvector(2)
    h(qubits[0])
    x.ctrl(qubits[0], qubits[1])

# Set the target to IonQ Aria 1
cudaq.set_target('ionq', qpu='qpu.aria-1')

# Submit the circuit
async_result = cudaq.sample_async(hello_qpu, shots_count=1000)

# Save the job information to a file
with open("hello_qpu.txt", "w") as file:
    file.write(str(async_result))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



RuntimeError: HTTP POST Error - status code 400: : {"statusCode":400,"error":"Bad Request","message":"Unable to run jobs on IonQ Aria. Please visit our Quantum Cloud Page https://ionq.com/quantum-cloud to learn more about getting access to our IonQ Aria."}

During handling of the above exception, another exception occurred:

AttributeError: 'RuntimeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

AssertionError
RuntimeError: HTTP POST Error - status code 400: : {"statusCode":400,"error":"Bad Request","message":"Unable to run jobs on IonQ Aria. Please visit our Quantum Cloud Page https://ionq.com/quantum-cloud to learn more about getting access to our IonQ Aria."}

During handling of the above exception, another exception occurred:

AttributeError: 'RuntimeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

TypeError: object of type 'NoneType' ha

In [1]:
import numpy as np
import cudaq
from cudaq import spin
from typing import List

# We'll use the graph below to illustrate how QAOA can be used to
# solve a max cut problem

#       v1  0--------------0 v2
#           |              | \
#           |              |  \
#           |              |   \
#           |              |    \
#       v0  0--------------0 v3-- 0 v4
# The max cut solutions are 01011, 10100, 01010, 10101 .

# First we define the graph nodes (i.e., vertices) and edges as lists of integers so that they can be broadcast into
# a cudaq.kernel.
nodes: List[int] = [0, 1, 2, 3, 4]
edges = [[0, 1], [1, 2], [2, 3], [3, 0], [2, 4], [3, 4]]
edges_src: List[int] = [edges[i][0] for i in range(len(edges))]
edges_tgt: List[int] = [edges[i][1] for i in range(len(edges))]